# <font color='blue'>Advanced Challenge: Randomized Benchmarking</font> 

In this challenge, you will implement randomised benchmarking, a protocol to determine the 
average gate fidelity of a quantum process.

You saw the basic idea in Ellen's talk a few days ago.

One of the first papers on this subject can be found [here](https://arxiv.org/abs/0707.0963).

This is a method which is 'easy' to implement in the near term, and gives an ideal about how well our
quantum device is performing. In this sense, it is a *lightweight* form of verification of quantum computation.

## <font color='red'>About this challenge</font> 

In this challenge, you can implement the randomized benchmarking protocol in the simplest case (RB of a single qubit).

### <font color='orange'>Part 1:</font> 

First, you can do it for the 'trivial' case, i.e. the case where our quantum gates have **no** errors (of course this is pointless, since it is exactly the point of RB to determine some error, which will be zero in this case!)
but it will show you how the protocol works.

### <font color='orange'>Part 2:</font> 

Then we can try and artificially introduce errors on each gate, and use RB to determine the average value of this error.

Firstly, import the usual things we need:

In [ ]:
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
import numpy as np
import os, inspect, sys
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, 'tests/')
sys.path.insert(0, 'auxiliary_functions/')

from auxiliary_functions.auxiliary_rb import *

%matplotlib inline

make_wf = WavefunctionSimulator()


We are only using a single qubit, but we will use a two qubit simulator for no particular reason.

In [ ]:
qc_name = '2q-qvm'
with local_qvm():
    qc = get_qc(qc_name)

qubits = qc.qubits()

We need to run RB for different sequences of gates, which have different lengths. We will want to get a 'datapoint' for each of these sequence lengths.

The longer the gate sequence, the more error is introduced, so it is less likely to find the qubit in the correct state at the end (*this will make more sense shortly*)

In [ ]:
#Firstly define the pulse lengths. This will be the number of gates in a particular run
    
lengths = [2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50, 80, 90, 100] 


### <font color='orange'>Back to Part 1:</font> 

## Randomized Benchmarking Protocol: Ideal Case

Here, we will apply noiseless gates (since we are using a perfect simulator), so the computation should be perfect.

## <font color='red'>Task:</font> 

Define a function which takes a list of the possible gates and creates a **list** of random gates of length 'seq_len', which will be applied to the zeroth qubit in the chip. 

The gates we apply will be from the [Clifford Group](https://en.wikipedia.org/wiki/Clifford_algebra#Clifford_group) for technical reasons. From the [Gottesman-Knill Theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem), this gate set can be simulated efficiently by a classical computer.

In [ ]:
import random

def random_clifford_noiseless_sequence(clifford_gates, seq_len):
    '''Generates a sequence of random gates from the set of noiy gates of length 'seq_len' '''   
    random_gate_sequence = []
 
    return random_gate_sequence

## <font color='red'>Task:</font> 

Next, define a function which takes in a Quil Program (a quantum circuit) and applies this sequence of gates, and then applies the inverse of every gate in the list.

In [ ]:
def apply_random_noiseless_gate_sequence(circuit, random_gate_sequence):
    '''Applies the random_gate_sequence of length 'seq_len' to the circuit'''
    
    circuit += #Apply sequence here
 
    return circuit


Finally, 

Now take that circuit of random cliffords, and measure the fidelity of the final state remaining in the initial state. 

The final state is denoted $|\psi\rangle$, and the initial state was the computational basis state, $|0\rangle$.

If the sequence was perfectly inverted, (which it will be in the perfect case), we get back exactly the state we started with (i.e. as if the identity was applied and nothing happened to the state - in the noisy case this will not be true)

We measure 'how much' of the original state is left by measuring the ***fidelity*** which is given by:

\begin{align}
F = |\langle 0 |\psi \rangle|^2 = \Pr(final state = |0\rangle)
\end{align}

So we can estimate this by measuring the qubit a number of times, and counting the number of times we get the '0' ($|0\rangle$) outcome.

## <font color='red'>Task:</font> 
### Apply a measurement on the single qubit and compute the fidelity.

In other words, compute the relative number of times we see the '0' outcome - the probability of getting 0

In [ ]:
def compute_fidelity_noiseless(circuit, random_gate_seq, length, num_shots):
    
    circuit = apply_random_noiseless_gate_sequence(circuit, random_gate_seq) # Apply the random sequence
  
    prob_zero = # Compute fidelity here 

    return prob_zero


## <font color='red'>Task:</font> 
### Finally, we will compute the average fidelity of the process, *per run*, *per sequence length*, by just running this many times

We need:

\begin{align}
1) &\text{ The number of measurement shots to measure the qubit in computing the fidelity **in each run** . }\\
2) &\text{ The number of times we want to run a sequence of gates of a particular length. }\\
3) &\text{ The number of sequence lengths we choose. }\\
\end{align}

*Note: You should not need to alter the below function*

In [ ]:
def compute_avg_fidelity_noiseless(lengths, num_shots, num_seq_repeats):
    
    average_fidelities = np.zeros(len(lengths)) # Array of average fidelities for each sequence length
    
    for length in range(len(lengths)):
        clifford_gates = define_cliffords_part1()# Define the list of possible Clifford gates (predefined for you)
        random_gate_seq = random_clifford_noiseless_sequence(clifford_gates, length) # Choose random sequence
        
        fidelity = np.zeros(num_seq_repeats)
        seq_len = lengths[length]
        
        for repeat in range(num_seq_repeats):
            circuit = Program()
            # Define noisy cliffords so they can be used in the circuit
            circuit = apply_random_noiseless_gate_sequence(circuit, random_gate_seq)

            fidelity[repeat] = compute_fidelity_noiseless(circuit, random_gate_seq, length, num_shots)
            print('Fidelity for run, ', repeat, 'is', fidelity[repeat])
            
        average_fidelities[length] = (1/num_seq_repeats)*np.sum(fidelity)
        print('Average fidelity for length: \n', lengths[length],'\n is:\n', average_fidelities[length])
        
    return average_fidelities

Ok, now choose the number of shots and number of sequence repeats:

In [ ]:
num_shots =  10
num_seq_repeats = 10

average_fidelities = compute_avg_fidelity_noiseless(lengths, num_shots, num_seq_repeats)

Next, we can plot the average fidelity as a function of the quantum circuit sequence lengths.
In the ideal case (that you have just done), clearly the average fidelity should be the same regardless of the sequence length.

When the inverse sequence of gates was applied, there was no noise, so effectively nothing happened to the $| 0\rangle$ state; it returned to the $|0\rangle$ state **every** time, and the fidelity should always be $1$.

Increasing the length of the circuit does not affect this.

We can plot the average fidelities as a function of the sequence lengths (this is a trivial task in this case, but it will become relevant shortly.)

In [ ]:
plt.plot(lengths, average_fidelities)

## <font color='orange'>Now onto to Part 2:</font> 
## Randomized Benchmarking Protocol: Noisy Case

We will pick a particular noise model to test. This model will apply a specific noise error to *each* gate we apply.

## <font color='red'>First things first..</font> 

Firstly, we must define 'noisy' gates to benchmark.

These will be noisy versions of the perfect clifford gates we used above.

For example, imagine our sequence of (one) random cliffords was $X$ applied the the input state:

\begin{align}
X|0\rangle
\end{align}

The corresponding density matrix would be (in the perfect case):

\begin{align}
\rho_{ideal} = X|0\rangle\langle 0|X
\end{align}

Now we can use the density matrix formalism to describe an ***error channel***. Firstly we can examine what is called a '*dephasing*' error channel, which means an erroneous phase error is applied to the state with some probability.

In other words, after we thought we had applied the $X$ gates to the state, because of the presence of this error channel, the *actual* state will be the correct state, with some probability, $p$, and some other state (with an error), with the remaining probability (1-p):

\begin{align}
\rho_{noisy} = \begin{cases}
\rho =  X|0\rangle\langle 0|X  &\text{ with some probability } p\\
Z\rho Z = ZX|0\rangle\langle 0|XZ &\text{ with probability } 1-p
\end{cases}
\end{align}

We can represent this by a *statistical mixture of these two possibilities*:
\begin{align}
\rho_{noisy} = \mathcal{E}_{dephase}(\rho_{ideal}) =  p\rho_{ideal}+ (1-p)Z\rho_{ideal} Z = pX|0\rangle\langle 0|X+ (1-p)ZX|0\rangle\langle 0|XZ 
\end{align}

where $\mathcal{E}_{dephase}(\rho_{ideal})$ is the dephasing error channel applied to the *ideal* state, $\rho_{ideal}$.

In the simplest example, we can assume this ***<font color='red'>same</font>*** error channel, $\mathcal{E}_{dephase}$  with the ***<font color='red'>same</font>***  error strength, $p$, is applied after ***<font color='blue'>every</font>***  random gate. 

We can then use apply the same protocol as above with the noisy clifford gates (with this dephasing error model)
to get an idea of the average fidelity of the process.

*Hint: This should **not** be $1$ for all sequence lengths in this case, we should see the fidelity decay as longer sequences are applied $\rightarrow$ the effect of noise compounds after each gate.*

Of course, when using the simulator, we must **artificially** define our noisy gates. We will directly use the example [here](http://docs.rigetti.com/en/stable/noise.html) to create the dephasing channel.

This uses the idea of Kraus maps, which is a generalisation of the above, for our purposes, a kraus map is just defined by a matrix, for example $Z$ or the identity $I$ (which would be the ideal case).

The above equation can be written as:

\begin{align}
\rho_{noisy} = \mathcal{E}_{dephase}(\rho_{ideal}) =  (\sqrt{p}\mathbb{I})(\rho_{ideal})(\sqrt{p}\mathbb{I})+ \sqrt{(1-p)}Z\rho_{ideal} \sqrt{(1-p)}Z 
\end{align}


## <font color='blue'>Challenge</font> 

Repeat the entire process above, but instead of applying the **perfect** clifford gates, we must define and implement the noisy versions, which are predefined for you.

## <font color='red'>Task</font> 

Define a function which takes a list of the possible clifford gates and creates a **list** of random gates of length 'seq_len', which will be applied to the zeroth qubit in the chip. 

*Note: We will not apply gates from the entire clifford group here, for this challenge it will not be relevant (we will see why shortly).
In general, we **do** need to apply gates from the entire Clifford group as in the perfect case, above.*


In [ ]:
import random

def random_clifford_noisy_sequence(noisy_clifford_gates, seq_len):
    '''Generates a sequence of random gates from the set of noisy gates of length 'seq_len' '''  
    random_noisy_gate_sequence = []
    
    random_noisy_gate_sequence = # Generate random list here from set of noisy gates
    
    return random_noisy_gate_sequence


## <font color='red'>Task:</font> 

Next, define a function which takes in a Quil Program (a quantum circuit) and applies this sequence of gates, and then applies the inverse of every gate in the list. This is different to the previous (noiseless) case, since in this case, we must define the strength of the error, $p$.

In [ ]:
def apply_random_noisy_gate_sequence(circuit, random_gate_sequence):
    '''Generates a sequence of random operations'''


    circuit += # Apply random sequence of noisy clifford gates and it's inverse here
    
    return circuit


## <font color='red'>Task:</font> 
### Apply a measurement on the single qubit and compute the fidelity.

Same as above in the noiseless case.

In other words, compute the relative number of times we see the '0' outcome - the probability of getting 0

In [ ]:
def compute_fidelity_noisy(circuit, random_gate_sequence, length, num_shots):
    
    circuit = apply_random_noisy_gate_sequence(circuit, random_gate_sequence)
    
    prob_zero = # Compute probability of getting zero here

    return prob_zero

## <font color='red'>Task:</font> 
### Finally, we will compute the average fidelity of the process, *per run*, *per sequence length*, by just running this many times, again just as in the ideal case

We need:

\begin{align}
1) &\text{ The number of measurement shots to measure the qubit in computing the fidelity **in each run** . }\\
2) &\text{ The number of times we want to run a sequence of gates of a particular length. }\\
3) &\text{ The number of sequence lengths we choose. }\\
3) &\text{ The strength of the noise we apply to each gate. }\\
\end{align}

*Note: You should not need to alter the below function, as above*

We can load the sequence of predefined noisy clifford gates from an auxiliary function:

In [ ]:
def compute_avg_fidelity_noisy(lengths, num_shots, num_seq_repeats, p=0.1):
    
    average_fidelities = np.zeros(len(lengths)) # Array of average fidelities for each sequence length
    
    for length in range(len(lengths)):
        clifford_gates = define_cliffords_part2()# Define list 
        noisy_clifford_gates = define_noisy_cliffords(p)
        # Generate a sequence of noisy gates
        random_gate_seq = random_clifford_noisy_sequence(clifford_gates, length) 

        fidelity = np.zeros(num_seq_repeats)
        seq_len = lengths[length]
        
        for repeat in range(num_seq_repeats):
            circuit = Program()
            # Define noisy cliffords so they can be used in the circuit
            circuit = add_noisy_gates_to_circ(circuit, noisy_clifford_gates)

            fidelity[repeat] = compute_fidelity_noisy(circuit, random_gate_seq, length, num_shots)
            print('Fidelity for run, ', repeat, 'is', fidelity[repeat])
            
        average_fidelities[length] = (1/num_seq_repeats)*np.sum(fidelity)
        print('Average fidelity for length: \n', lengths[length],'\n is:\n', average_fidelities[length])
    return average_fidelities

num_seq_repeats = 10# Number of repeats of a particular sequence length

num_shots = 10

average_fidelities = compute_avg_fidelity_noisy( lengths, num_shots, num_seq_repeats, p=0.1)


In [ ]:
plt.plot(lengths, average_fidelities)

Oh no! What happened here? For some reason the fidelity was still $1$ for all sequence lengths...

What do you think was the cause?

*Hint: Have a look at the compiled code produced by the quil compiler. What gates were actually applied to the qubit?*